In [ ]:
! nvidia-smi

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sys
from nltk.tokenize import TweetTokenizer
from emoji import demojize

import random
import math

from tqdm.notebook import tqdm
import string

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold

import tensorflow as tf
from tensorflow.keras import backend as K

import transformers
from transformers import TFAutoModel, AutoTokenizer, TFAutoModelForSequenceClassification

from collections import Counter

print('Using Tensorflow version:', tf.__version__)
print('Using Transformers version:', transformers.__version__)

# import warnings
# warnings.filterwarnings('ignore')
tqdm.pandas()

In [ ]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
seed = 1512
def seed_all(seed=1512):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
seed_all(seed)

In [ ]:
# %%time

train_df = pd.read_csv("../data/tokenized_data/train_5_folds.csv")
test_df = pd.read_csv("../data/tokenized_data/test.csv")

train_df["post_message"] = train_df["post_message"].astype(str)
test_df["post_message"] = test_df["post_message"].astype(str)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
roberta = 'vinai/phobert-base' 
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta)
roberta_model = TFAutoModel.from_pretrained(roberta)

In [ ]:
# train_len_word = [len(text.split()) for text in train_df.post_message]
# test_len_word = [len(text.split()) for text in test_df.post_message]
# test_len_char = [len(text) for text in train_df.post_message]
# test_len_char = [len(text) for text in test_df.post_message]

In [ ]:
# def length_plot(lengths):
#     plt.figure(figsize=(12,7))
#     textstr = f' Mean: {np.mean(lengths):.2f} \u00B1 {np.std(lengths):.2f} \n Max: {np.max(lengths)}'
#     props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

#     plt.text(0, 0, textstr, fontsize=14,
#             verticalalignment='top', bbox=props)
#     sns.countplot(lengths)    

In [ ]:
# length_plot(train_len_word)

In [ ]:
# length_plot(test_len_word)

In [ ]:
MAX_LEN = 256
BATCH_SIZE = 48

In [ ]:
def regular_encode(texts, maxlen=MAX_LEN):

    roberta_enc_di = roberta_tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )
    
    roberta_enc = (
        np.array(roberta_enc_di["input_ids"]),
        np.array(roberta_enc_di["attention_mask"]),
        np.array(roberta_enc_di["token_type_ids"]),
    )
    return roberta_enc

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

def data_generator(train_df, val_df):

    X_train = regular_encode(train_df["post_message"].values, maxlen=MAX_LEN)
    # y_train = tf.keras.utils.to_categorical(train_df['Label'].values, num_classes=2)
    y_train = train_df["label"].values
    X_val = regular_encode(val_df["post_message"].values, maxlen=MAX_LEN)
    # y_val = tf.keras.utils.to_categorical(val_df['Label'].values, num_classes=2)
    y_val = val_df["label"].values

    train_dataset = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .repeat()
        .shuffle(1024)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )

    return train_dataset, valid_dataset

In [ ]:
def build_model(bert_model_name_or_path="vinai/phobert-base", max_len=384, n_hiddens=4):
    bert_model = TFAutoModel.from_pretrained(bert_model_name_or_path)

    bert_input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_input_id"
    )
    bert_attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_attention_mask"
    )
    bert_token_type_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_token_type_ids"
    )

    bert_sequence_output = bert_model(
        bert_input_word_ids,
        attention_mask=bert_attention_mask,
        token_type_ids=bert_token_type_ids,
        output_hidden_states=True,
        output_attentions=True,
        
    )

    # print(len(bert_sequence_output)) # 4

    # print(bert_sequence_output[0].shape) # (None, max_len, 768)

    # print(bert_sequence_output[1].shape) # (None, 768)
    # print(len(bert_sequence_output[2])) # 13
    # print(bert_sequence_output[2][0].shape) # (None, max_len, 768)
    # print(len(bert_sequence_output[3])) # 12
    # print(bert_sequence_output[3][0].shape) # (None, 12, None, max_len)

    # TODO: get bert embedding

    if n_hiddens == -1:  # get [CLS] token embedding only
        # print("Get pooler output of shape (batch_size, hidden_size)")
        bert_sequence_output = bert_sequence_output[0][:, 0, :]
    else:  # concatenate n_hiddens final layer
        # print(f"Concatenate {n_hiddens} hidden_states of shape (batch_size, hidden_size)")
        bert_sequence_output = tf.concat(
            [bert_sequence_output[2][-i] for i in range(n_hiddens)], axis=-1)

    # print("bert_sequence_output shape", bert_sequence_output.shape)

    
    
    out = tf.keras.layers.Flatten()(bert_sequence_output)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(out)

    model = tf.keras.models.Model(
        inputs=[
            bert_input_word_ids,
            bert_attention_mask,
            bert_token_type_ids,  # bert input
        ],
        outputs=out,
    )
    model.compile(
        tf.keras.optimizers.Adam(lr=5e-5),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC()],
    )

    return model

In [ ]:
%%time
model = build_model(max_len=MAX_LEN)
model.summary()

In [ ]:
exp = 'roberta-baseline' 
n_splits = 5
n_epochs = 5

DISPLAY=1 # USE display=1 FOR INTERACTIVE
exp = f'phobert_{MAX_LEN}_len'

output_dir = f'../outputs/{exp}_models'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def scheduler(epoch):
    return 3e-5*0.2**epoch

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
# # for fold, (idxT, idxV) in enumerate(kf.split(train_df)):
# for fold in sorted(train_df["fold"].unique()):
#     print('*'*100)
#     print(f'FOLD: {fold+1}/{n_splits}')
#     if fold<4:
#         continue
#     print('*'*100)
#     print(f'FOLD: {fold+1}/{n_splits}')
    
#     K.clear_session()
#     with strategy.scope():
#         model = build_model(max_len=MAX_LEN)
        
#     reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

#     model_dir = os.path.join(output_dir, f'Fold_{fold+1}.h5')

#     sv = tf.keras.callbacks.ModelCheckpoint(model_dir, 
#                                             monitor='val_auc', 
#                                             verbose=1, 
#                                             save_best_only=True,
#                                             save_weights_only=True, 
#                                             mode='max', 
#                                             save_freq='epoch')
    
#     train_df_ = train_df[train_df["fold"]!=fold]
#     val_df_ = train_df[train_df["fold"]==fold]
#     train_dataset, valid_dataset = data_generator(train_df_, val_df_)
    
#     n_steps = train_df_.shape[0] // BATCH_SIZE
#     train_history = model.fit(
#         train_dataset,
#         steps_per_epoch=n_steps,
        
#         callbacks=[sv, 
#             reduce_lr,
#             # tb
#             ],
#         validation_data=valid_dataset,
#         epochs=n_epochs
#     )

In [ ]:
# model.predict(valid_dataset)

In [ ]:
# X_test = regular_encode(test_df['post_message'].values, maxlen=MAX_LEN)
# y_test = np.zeros((len(test_df),1))
# test_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((X_test,y_test))
#     .batch(BATCH_SIZE)
# )

In [ ]:
# model = build_model(max_len=MAX_LEN)
# preds = []
# for i, file_name in enumerate(os.listdir(output_dir)):
#     print('_'*80)
    
#     K.clear_session()
#     model_path = os.path.join(output_dir, file_name)
    
#     print(f'Inferencing with model from: {model_path}')
#     model.load_weights(model_path)

#     pred = model.predict(test_dataset,
#                          batch_size=128,
#                          verbose=DISPLAY)
#     # print(pred[])
#     preds.append(pred)

In [ ]:
# preds = np.mean(preds, axis=0)
# test_df["prediction"] = preds
# test_df["prediction"].to_csv(f"{exp}.csv", header=False)

### EDA

In [ ]:
train_df.head()

In [ ]:
test_df.head(10)

In [ ]:
Counter(train_df["num_like_post"])["unknown"]

In [ ]:
sum(train_df["num_like_post"].isna())

In [ ]:
Counter(train_df["num_share_post"])["unknown"]

In [ ]:
sum(train_df["num_share_post"].isna())

In [ ]:
Counter(train_df["num_comment_post"])["unknown"]

In [ ]:
sum(train_df["num_comment_post"].isna())

In [ ]:
nan_dict = {"num_like_post":-2,"num_comment_post":-2, "num_share_post":-2}
unknown_dict = {"num_like_post":-2,"num_comment_post":-2, "num_share_post":-2}

def extract_num(text):
    return re.findall("\d+", text)[0]
def vlsp_impute(text, field):
    if type(text)!=str:
        if math.isnan(text):
            return nan_dict[field]
    elif text=="unknown":        
        return unknown_dict[field]
    else:
        try:
            return int(extract_num(text))
        except:
            print(text)
            return nan_dict[field]

In [ ]:
for field in tqdm(["num_like_post", "num_comment_post", "num_share_post"]):
#     with open(f"train_{field}.txt", "w") as f:
#         f.write("\n".join([str(x) for x in train_df[field].unique()]))
#     with open(f"test_{field}.txt", "w") as f:
#         f.write("\n".join([str(x) for x in test_df[field].unique()]))    
    train_df[field] =  train_df[field].apply(lambda text: vlsp_impute(text, field)).astype(int)
    test_df[field] =  test_df[field].apply(lambda text: vlsp_impute(text, field)).astype(int)

In [ ]:
def regular_encode(df, max_len=MAX_LEN):
    
    roberta_enc_di = roberta_tokenizer.batch_encode_plus(
        df["post_message"].values,
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=max_len,
        truncation=True,
    )
    
    num_like_post = (df["num_like_post"]/1e6).values
    num_comment_post = (df["num_comment_post"]/1e4).values
    num_share_post = (df["num_share_post"]/1e4).values
    
    roberta_enc = (
        np.array(roberta_enc_di["input_ids"]),
        np.array(roberta_enc_di["attention_mask"]),
        np.array(roberta_enc_di["token_type_ids"]),
        num_like_post,
        num_comment_post,
        num_share_post,
#         np.concatenate([num_like_post, num_comment_post, num_share_post], axis=0)
    )
    
    
    return roberta_enc

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

def data_generator(train_df, val_df):

    X_train = regular_encode(train_df, max_len=MAX_LEN)
    # y_train = tf.keras.utils.to_categorical(train_df['Label'].values, num_classes=2)
    y_train = train_df["label"].values
    X_val = regular_encode(val_df, max_len=MAX_LEN)
    # y_val = tf.keras.utils.to_categorical(val_df['Label'].values, num_classes=2)
    y_val = val_df["label"].values

    train_dataset = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .repeat()
        .shuffle(1024)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )

    return train_dataset, valid_dataset

In [ ]:
def build_model(bert_model_name_or_path="vinai/phobert-base", max_len=384, n_hiddens=-1):
    bert_model = TFAutoModel.from_pretrained(bert_model_name_or_path)

    bert_input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_input_id"
    )
    bert_attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_attention_mask"
    )
    bert_token_type_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_token_type_ids"
    )

    bert_sequence_output = bert_model(
        bert_input_word_ids,
        attention_mask=bert_attention_mask,
        token_type_ids=bert_token_type_ids,
        output_hidden_states=True,
        output_attentions=True,
        
    )

    # print(len(bert_sequence_output)) # 4

    # print(bert_sequence_output[0].shape) # (None, max_len, 768)

    # print(bert_sequence_output[1].shape) # (None, 768)
    # print(len(bert_sequence_output[2])) # 13
    # print(bert_sequence_output[2][0].shape) # (None, max_len, 768)
    # print(len(bert_sequence_output[3])) # 12
    # print(bert_sequence_output[3][0].shape) # (None, 12, None, max_len)

    # TODO: get bert embedding

    if n_hiddens == -1:  # get [CLS] token embedding only
        # print("Get pooler output of shape (batch_size, hidden_size)")
        bert_sequence_output = bert_sequence_output[0][:, 0, :]
#         bert_sequence_output = bert_sequence_output[1]
    else:  # concatenate n_hiddens final layer
        # print(f"Concatenate {n_hiddens} hidden_states of shape (batch_size, hidden_size)")
        bert_sequence_output = tf.concat(
            [bert_sequence_output[2][-i] for i in range(n_hiddens)], axis=-1)
        bert_sequence_output = bert_sequence_output[:, 0, :]

    # print("bert_sequence_output shape", bert_sequence_output.shape)

#     bert_output = tf.keras.layers.Flatten()(bert_sequence_output)
    bert_output = tf.keras.layers.Dense(16, activation="relu")(bert_sequence_output)
#     print(bert_output.shape)
    

    num_like_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_like_post")
    num_comment_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_comment_post")
    num_share_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_share_post")
    
    aulixiary_info = tf.keras.layers.Concatenate()([num_like_post, num_comment_post, num_share_post])
#     aulixiary_output  = tf.keras.layers.GaussianNoise(0.2)(aulixiary_info)
    
    out = tf.keras.layers.Concatenate()([bert_output, aulixiary_info]) 
#     print(out.shape)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(out)

    model = tf.keras.models.Model(
        inputs=[
            bert_input_word_ids,
            bert_attention_mask,
            bert_token_type_ids,  # bert input
            num_like_post,
            num_comment_post,
            num_share_post,
        ],
        outputs=out,
    )
    model.compile(
        tf.keras.optimizers.Adam(lr=5e-5),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC()],
    )

    return model

In [ ]:
%%time
model = build_model(max_len=MAX_LEN)
model.summary()

In [ ]:
n_splits = 5
n_epochs = 5

DISPLAY=1 # USE display=1 FOR INTERACTIVE
exp = f'phobert+auxiliary_{MAX_LEN}_len'

output_dir = f'../outputs/{exp}_models'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def scheduler(epoch):
    return 3e-5*0.2**epoch

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
# for fold, (idxT, idxV) in enumerate(kf.split(train_df)):
for fold in sorted(train_df["fold"].unique()):
    print('*'*100)
    print(f'FOLD: {fold+1}/{n_splits}')
    
    K.clear_session()
    with strategy.scope():
        model = build_model(max_len=MAX_LEN)
        
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    model_dir = os.path.join(output_dir, f'Fold_{fold+1}.h5')

    sv = tf.keras.callbacks.ModelCheckpoint(model_dir, 
                                            monitor='val_auc', 
                                            verbose=1, 
                                            save_best_only=True,
                                            save_weights_only=True, 
                                            mode='max', 
                                            save_freq='epoch')
    
    train_df_ = train_df[train_df["fold"]!=fold]
    val_df_ = train_df[train_df["fold"]==fold]
    train_dataset, valid_dataset = data_generator(train_df_, val_df_)
    
    n_steps = train_df_.shape[0] // BATCH_SIZE
    train_history = model.fit(
        train_dataset,
        steps_per_epoch=n_steps,
        
        callbacks=[sv, 
            reduce_lr,
            # tb
            ],
        validation_data=valid_dataset,
        epochs=n_epochs
    )

In [ ]:
X_test = regular_encode(test_df, max_len=MAX_LEN)
y_test = np.zeros((len(test_df),1))
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test,y_test))
    .batch(BATCH_SIZE)
)

In [ ]:
model = build_model(max_len=MAX_LEN)
preds = []
for i, file_name in enumerate(os.listdir(output_dir)):
    print('_'*80)
    
    K.clear_session()
    model_path = os.path.join(output_dir, file_name)
    
    print(f'Inferencing with model from: {model_path}')
    model.load_weights(model_path)

    pred = model.predict(test_dataset,
                         batch_size=128,
                         verbose=DISPLAY)
    # print(pred[])
    preds.append(pred)

In [ ]:
preds = np.mean(preds, axis=0)
test_df["prediction"] = preds
test_df["prediction"].to_csv(f"{exp}.csv", header=False)